##  Retrieval Augmented Generation

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

In [3]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader

In [4]:
document = SimpleDirectoryReader("data").load_data()

In [5]:
document

[Document(id_='f206a6e1-d86e-44ca-8afd-43a52cf45c2e', embedding=None, metadata={'page_label': '1', 'file_name': 'd:/Nikku/End to End Project/RAG/data/attention.pdf', 'file_path': 'd:\\Nikku\\End to End Project\\RAG\\data\\attention.pdf', 'file_type': 'application/pdf', 'file_size': 569417, 'creation_date': '2024-03-01', 'last_modified_date': '2024-03-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\n

In [6]:
# Covert this into index
index = VectorStoreIndex.from_documents(document,show_progress=True)

d:\Nikku\End to End Project\RAG\venvrag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 71/71 [00:01<00:00, 35.78it/s]


In [7]:
index

In [8]:
query_engine = index.as_query_engine()

In [9]:
print(query_engine.query("What is Transformers"))

Transformers are a new simple network architecture proposed in the context that is based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. They allow for significantly more parallelization and can achieve state-of-the-art results in tasks like machine translation. The Transformer model relies entirely on self-attention to compute representations of its input and output without using sequence-aligned recurrent neural networks or convolutional neural networks.


In [10]:
# from llama_index.response.pprint_utils import pprint_response

In [11]:
print(query_engine.query("What is Yolo and Transformers"))

Yolo is a real-time object detection system that processes images in a single pass. Transformers are a type of neural network architecture that has been successful in various natural language processing tasks.


In [12]:
from llama_index.core.retrievers import  VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.indices.postprocessor import SimilarityPostprocessor

In [14]:
retriever = VectorIndexRetriever(index=index,similarity_top_k=4)

query_engine = RetrieverQueryEngine(retriever=retriever)

In [18]:
from llama_index.core.response.pprint_utils import pprint_response

In [20]:
response = query_engine.query("What is Yolo and Transformers")

In [21]:
pprint_response(response,show_source=True)


Final Response: Yolo is a unified real-time object detection system
that focuses on processing images for object detection. Transformers,
on the other hand, are a type of neural network architecture that has
been widely used in natural language processing tasks.
______________________________________________________________________
Source Node 1/4
Node ID: 3e110c85-89a3-45cf-9e24-a0a06b6bfa95
Similarity: 0.7581746064454127
Text: https://www.slideshare.net/TaegyunJeon1/pr12-you-only-look-once-
yolo-unified-realtime-object-detection?from_action=save
______________________________________________________________________
Source Node 2/4
Node ID: ec6df464-981c-4db2-9077-85744453646c
Similarity: 0.7558843353772415
Text: https://www.slideshare.net/TaegyunJeon1/pr12-you-only-look-once-
yolo-unified-realtime-object-detection?from_action=save
______________________________________________________________________
Source Node 3/4
Node ID: a6f7dc72-7d83-4120-8037-13708ca068e2
Similarity: 0.75575618

In [26]:
postprocesser = SimilarityPostprocessor(similarity_cutoff=0.70)

In [27]:
query_engine = RetrieverQueryEngine(retriever=retriever,node_postprocessors=[postprocesser])


In [28]:
response = query_engine.query("What is Yolo")
pprint_response(response,show_source=True)


Final Response: Yolo stands for "You Only Look Once" and it is a
unified real-time object detection system.
______________________________________________________________________
Source Node 1/4
Node ID: 3e110c85-89a3-45cf-9e24-a0a06b6bfa95
Similarity: 0.7786190421342655
Text: https://www.slideshare.net/TaegyunJeon1/pr12-you-only-look-once-
yolo-unified-realtime-object-detection?from_action=save
______________________________________________________________________
Source Node 2/4
Node ID: b9d24ca6-64c0-4d00-893e-3914189841d2
Similarity: 0.7760938611565741
Text: YOLO:   You Only Look Once   Unified Real-Time Object Detection
Presenter: Liyang Zhong  Quan Zou
______________________________________________________________________
Source Node 3/4
Node ID: fc3e08d4-c2ad-4ba2-9209-f4d2dd666b44
Similarity: 0.775905899426715
Text: https://www.slideshare.net/TaegyunJeon1/pr12-you-only-look-once-
yolo-unified-realtime-object-detection?from_action=save
___________________________________________

In [32]:
import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage
)

# CHECK if stoarge aleary exicts
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    
    #load the documents and create the index
    documents =  SimpleDirectoryReader("data").load_data()
    
    index = VectorStoreIndex.from_documents(documents)
    
    #store it for later
    c
    index.storage_context.persist(persist_dir=PERSIST_DIR)

else:
    
    # Load the existing index
    stoarge_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(stoarge_context)

# Either way we can now query the index
query_engine =  index.as_query_engine()
response = query_engine.query("What are transformers?")

print(response)


Transformers are a model architecture that relies entirely on attention mechanisms to draw global dependencies between input and output, without using recurrence or convolutions. They allow for more parallelization and have shown superior performance in quality compared to traditional models in tasks such as machine translation.
